## Análisis de Simulated Annealing para una fuerza de ventas de 5 nodos

Simulated Annhealing es un algoritmo que recibe los siguientes hiperárametros de entrada: 

+ Tmax
+ Tmin 
+ steps 
+ updates 

Los resultados obtenidos pueden verse afectados al variar los valores de tales hiperparámetros. Por otro lado, dependiendo del número de nodos del grafo, estos hiperparámetros también podrían afectar la ruta mínima encontrada por el algoritmo.    

El presente notebook considera la implementación de simulated annhealing para un grafo con 5 nodos. Como primero objetivo se variarán los hiperparámetros y se identificarán aquellos que den mejores resultados. Entendiéndose como mejores resultados; obtener la ruta con menor distancia. Adicionalmente, una vez seleccionados los mejores hiperparámetros, se correrá el algoritmo 100 veces con el fin de realizar un análisis sobre las rutas obtenidas. Particularmente se tiene interés en revisar variaciones en las rutas obtenidas en cada corrida.

Dentro del conjunto de datos que se tienen disponibles, existen varias fuerzas de venta que deben recorrer 5 nodos. Se decidió elegir la fuerza de venta **80993** perteneciente al estado de Nuevo León para llevar a cabo estas pruebas.

In [1]:
!pip install dynaconf
!pip install psycopg2-binary
!pip install simanneal

In [2]:
# Librerías

import pandas as pd
import sys
sys.path.append('../')
%load_ext autoreload
%autoreload 2

from src import Utileria as ut
from src.models import particle_swarm as ps
from src.models import simulated_annealing as sa

### 1. Búsqueda de mejores hiperparámetros

**1.1 Definición de datos de entrada**
+ Grafo completo de los puntos que debe vistar la fuerza de ventas
+ Hiperámetros con los que correrá el algoritmo

In [3]:
# Se obtiene el dataframe que contiene el grafo de la fuerza de venta a evaluar:
str_Query = 'select id_origen, id_destino, distancia from trabajo.grafos where id_fza_ventas={};'

# En el query se especifica el id_fza_venta del cual se quiere obtener su grafo
df_Grafo = ut.get_data(str_Query.format(80993))
df_Grafo

Selecting rows from table using cursor.fetchall
PostgreSQL connection is closed


,id_origen,id_destino,distancia
0,11037,1001402004,0.267770992210818
1,11037,1006681965,0.3250713086280114
2,11037,1020053072,2.3599263518563016
3,11037,1020235635,1.2662382942527688
4,11037,1020402992,0.07674597387253217
5,1001402004,1006681965,0.4982193543875845
6,1001402004,1020053072,2.611292644038669
7,1001402004,1020235635,1.5321611628202725
8,1001402004,1020402992,0.23296305988637042
9,1006681965,1020053072,2.141257356651251


In [19]:
# Se crea el diccionario de hiper-parámetros que se evaluarán

#Default parameters
#Tmax = 25000.0  # Max (starting) temperature
#Tmin = 2.5      # Min (ending) temperature
#steps = 50000   # Number of iterations
#updates = 100   # Number of updates (by default an update prints to stdout)

dict_Hiper_SA = {'Tmax': {1000,10000, 25000.0, 50000},
              'Tmin': {1,2.5,5,10},
              'steps': {500,5000,10000},
              'updates': {10,50,100,200}
              }



**1.2 Gridsearch**

Dentro de la clase Utileria fue definido un método llamado *GridSearch*, el cual recibe como parámetros el grafo de una fuerza de ventas fijo, un diccionario de parámetros, el algoritmo a evaluar y el número de iteraciones que se correrá por cada combinación de hiperámetros. Este método evalúa el algoritmo con todas las combinaciones que se pueden generar a partir del diccionario de parámetros. En este caso se considerarán 3 valores de *Iteraciones*, 3 valores del *Número de Partículas*, 2 valores de $\alpha$ y 2 de $\beta$; dando lugar a un total de 36 combinaciones. Cada combinación de hiperarámetros se correrá 100 veces y como resultado se obtendrá una tabla indicando los Hiperámetros utilizados, las distancias mínima y máxima obtenidas dentro de las 100 corridas; y el número de corridas en que se repitió tal distancia mínima.

In [ ]:
%%time

# Se corre el GridSearch para el grafo y los hiperparámetros previamente definidos

df_Resultado = ut.GridSearch(df_Grafo, sa.SimulatedAnnealing, dict_Hiper_SA, 100)

 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.18%     0.02%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.90%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.07%     0.02%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.90%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.49%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.05%     0.00%     0:00:01    -1:59:59 Temperature 

     0.26000          4.00    67.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    66.40%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    70.70%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.24%     0.02%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    69.05%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.15%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.90%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000 

     0.26000          4.00    67.45%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.92%     0.04%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    67.00%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    67.17%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    67.30%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    67.98%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.90%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000 

     0.26000          4.00    67.40%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.16%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.95%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.25%     0.04%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.30%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.53%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.95%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000 

     0.26000          4.00    68.30%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.08%     0.08%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.05%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    67.97%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.15%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.71%     0.08%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.55%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000 

...

     0.17000          4.00    67.85%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.39%     0.00%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    66.40%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.85%     0.05%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.13%     0.01%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.30%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000 

     0.26000          4.00    68.00%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.85%     0.01%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.65%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.88%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.95%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.15%     0.03%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    66.95%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000 

     0.26000          4.00    69.30%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.76%     0.00%     0:00:08     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    66.75%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.12%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    66.00%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.26%     0.03%     0:00:08     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000 

     0.26000          4.00    67.75%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.24%     0.00%     0:00:08     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.60%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    66.55%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    69.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.17000          4.00    68.47%     0.00%     0:00:06     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000 

     0.26000          4.00    68.00%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.83%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    70.57%     0.04%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    69.75%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    67.02%     0.00%     0:00:07     0:00:00 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000          4.00    68.20%     0.00%     0:00:01    -1:59:59 Temperature        Energy    Accept   Improve     Elapsed   Remaining
     0.26000 

In [18]:
# Se muestra el dataframe con los resultados obtenidos de la corrida del GridSearch
pd.options.display.max_colwidth = 100
df_Resultado

,HiperParámetros,Distancia mínima (km),Distancia máxima (km),Frec. rel. dist. min.
0,"{'Tmax': 2, 'Tmin': 1, 'steps': 1, 'updates': 1}",4,4,1/1


**1.3 Análisis y Resultados**

En primer lugar es importante mencionar las motivaciones de ciertos de los hiperparámetros elegidos. 

+ Con respecto al número de iteraciones, $100$ iteraciones fueron elegidas con base en pruebas realizadas en la literatura [](), tomando este número como base, se determinó hacer pruebas considerando la mitad de iteraciones ($50$); y finalmente para tener un valor extremo contra el cuál comparar se eligió dismniur el número de iteraciones incial en un $90%$, dando lugar a $10$ iteraciones.
+ Sobre el número de partículas, la primera idea fue consideran tantas partículas como número de nodos, en este caso $5$; y tomar un número de partículas considerablemente mayor ($100$) y uno considerablemente menor ($2$). En este caso, al tener únicamente $5$ nodos no es posible tomar un número menor de partículas extremo; sin embargo los resultados obtenidos con $1$ partícula aportan resultados muy relevantes para el análisis.
+ Puesto que $\alpha$ y $\beta$ representan probabilidades, los valores que pueden tomar están entre $0$ y $1$. De manera análoga a los otros parámetros, se consideraron valores bajos($0$), medios($0.5$) y altos($1$).

Para poder interpretar los resultados mostrados en el dataframe anterior es conveniente recordar que se realizaron 36 pruebas, es decir, se tuvieron 36 combinaciones de hiperparámetros; y cada una de esas combinaciones se corrió 100 veces, dando un total de 3,600 corridas. En las 100 corridas de cada prueba se registró la distancia mínima obtenida, la distancia máxima y la frecuencia relativa de la distancia mínima. A continuación se enlistan observaciones importantes derivadas de estas pruebas:

+ La distancia mínima obtenida: $5.604 km$ es la misma para las 36 combinaciones; sin embargo la distancia máxima obtenida varía entre $5.604 km$ y $8.559 km$. Esto demuestra que apesar de utilizar los mismos hiperparámetros, el algoritmo puede dar resultados distintos en cada corrida.
+ Al considerar únicamente $1$ partícula y variar el resto de los hiperparámetros, se observa que la frecuencia relativa de la distancia mínima en cada una de las pruebas ($100$ corridas por prueba) es menor al $5\%$. 
+ Al fijar el número de partículas en $100$ y variar el resto de los hiperparámetros, se obtienen frecuencias relativas de la distancia mínima mayores al $90\%$. Éstas corresponden a las frecuencias relativas más altas de las 36 pruebas realizadas.
+ En general, al fijar tanto el número de iteraciones como el número de partículas y variar los valores de $\alpha$ y $\beta$, se observa que bajo la siguiente condición $\alpha = \beta = 1$ se obtiene los valores más chicos de frecuencias relativas de la distancia mínima. Al considerar estos parámetros igual a $1$ se está considerando que ambos influyen "totalmente" en la actualización de la posición de cada una de las partículas; ya que tanto $\alpha$ como $\beta$ pueden tomar valores entre $[0,1]$, representando una probabilidad.
+ Con respecto al número de iteraciones, puede decirse que este parámetro depende en gran medida del número de partículas. Por un lado, al considerar $100$ iteraciones y $1$ ó $5$ partículas, las frecuencias relativas de la distancia mínima son bajas. Por otro lado, al disminuir el número de iteraciones a $10$ y aumentar el número de partículas a $100$, tales frecuencias relativas son las más altas. 

### 2. Análisis de rutas

In [12]:
rutas = pd.DataFrame(index=range(3),columns=['km', 'Ruta'])

# Definición de hiperparámetros
dict_Hiper = {'Iteraciones': 10,
              'Particulas': 100,
              'Alfa': .5,
              'Beta': .5
              }

In [13]:
rutas

,km,Ruta
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN


In [14]:
for corrida in range(3):
    PS = ps.ParticleSwarm(df_Grafo,dict_Hiper)
    PS.Ejecutar()
    
    min_distancia = round(PS.nbr_MejorCosto,3)
    mejor_ruta = PS.lst_MejorCamino
    
    rutas.km[corrida] = min_distancia
    rutas.Ruta[corrida] = mejor_ruta
    
    

In [15]:
rutas

,km,Ruta
0,5.604,"[1020235635, 1020053072, 1006681965, 1001402004, 1020402992, 11037]"
1,5.604,"[1020402992, 1001402004, 1006681965, 1020053072, 1020235635, 11037]"
2,5.604,"[1020053072, 1006681965, 1001402004, 1020402992, 11037, 1020235635]"


### Conclusiones:
+ Los resultados del algoritmo pueden variar en cada corrida, aún cuando se mantengan fijos los hiperparámetros.
+ Correr el algoritmo una sola vez no garantiza que se obtenga la distancia mínima.
+ Al considerar un número de partículas alrededor de $100$ se tiene mayor probabilidad de encontrar la distancia mínima al correr una sola vez el algoritmo.
+ 


In [11]:
def convert(ruta): 
    s = [str(i) for i in ruta] 
    ruta_c = "-".join(s)
    return(ruta_c) 